In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

# 1. DATA GENERATION — MODEL I
np.random.seed(0)
n_samples = 50
n_factors = 15
rho = 0.5

# Covariance matrix for correlated normal variables
cov_matrix = rho ** np.abs(np.subtract.outer(np.arange(n_factors), np.arange(n_factors)))
Z = np.random.multivariate_normal(mean=np.zeros(n_factors), cov=cov_matrix, size=n_samples)

# Trichotomize Z into 0, 1, 2 categories
Z_discrete = np.zeros_like(Z, dtype=int)
for i in range(n_factors):
    q1, q2 = np.percentile(Z[:, i], [33.33, 66.67])
    Z_discrete[:, i] = np.digitize(Z[:, i], bins=[q1, q2])

# Response generation rule from paper
def simulate_response(Zd):
    y = (
        1.8 * (Zd[:, 0] == 1) - 1.2 * (Zd[:, 0] == 0) +
        1.0 * (Zd[:, 2] == 1) + 0.5 * (Zd[:, 2] == 0) +
        1.0 * (Zd[:, 4] == 1) + 1.0 * (Zd[:, 4] == 0)
    )
    noise = np.random.normal(0, np.std(y), size=y.shape)
    return y + noise

Y = simulate_response(Z_discrete)

# One-hot encode: drop one category to avoid multicollinearity
encoder = OneHotEncoder(drop='first', sparse_output=False)
X = encoder.fit_transform(Z_discrete)

# Centering
X -= X.mean(axis=0)
Y -= Y.mean()

# Group structure (each factor becomes 2 dummies after one-hot)
group_sizes = [2] * n_factors
group_indices = []
start = 0
for size in group_sizes:
    group_indices.append(np.arange(start, start + size))
    start += size

# 2. METHODS

# --- OLS Full Model ---
def model_ols(X, Y):
    model = LinearRegression(fit_intercept=False).fit(X, Y)
    beta = model.coef_
    return beta, X @ beta

# --- Group LARS (greedy approximation) ---
def group_lars(X, y, groups, max_groups=None):
    residual = y.copy()
    selected_groups = []
    beta = np.zeros(X.shape[1])

    for _ in range(max_groups or len(groups)):
        correlations = [
            np.linalg.norm(X[:, g].T @ residual) / len(g) if j not in selected_groups else -np.inf
            for j, g in enumerate(groups)
        ]
        best_group = np.argmax(correlations)
        if correlations[best_group] == -np.inf:
            break
        selected_groups.append(best_group)
        X_sub = np.hstack([X[:, g] for j, g in enumerate(groups) if j in selected_groups])
        model = LinearRegression(fit_intercept=False).fit(X_sub, y)
        coef = model.coef_
        index = 0
        for j in selected_groups:
            g = groups[j]
            beta[g] = coef[index:index + len(g)]
            index += len(g)
        residual = y - X @ beta
    return beta, X @ beta

# --- Group Non-negative Garrote (approximation) ---
def group_garrote(X, y, groups, lam=0.1):
    ols = LinearRegression(fit_intercept=False).fit(X, y)
    beta_ols = ols.coef_
    d = np.ones(len(groups))
    for j, g in enumerate(groups):
        group_norm = np.linalg.norm(beta_ols[g])
        if group_norm > 0:
            shrink = max(0, 1 - lam * len(g) / group_norm)
            d[j] = shrink
        else:
            d[j] = 0
    beta_final = np.zeros_like(beta_ols)
    for j, g in enumerate(groups):
        beta_final[g] = beta_ols[g] * d[j]
    return beta_final, X @ beta_final

# 3. APPLY METHODS
beta_ols, pred_ols = model_ols(X, Y)
beta_lars, pred_lars = group_lars(X, Y, group_indices, max_groups=5)
beta_garrote, pred_garrote = group_garrote(X, Y, group_indices, lam=0.5)

# 4. COMPARE MODELS
mse_ols = mean_squared_error(Y, pred_ols)
mse_lars = mean_squared_error(Y, pred_lars)
mse_garrote = mean_squared_error(Y, pred_garrote)

# Results table
results_df = pd.DataFrame({
    "Method": ["OLS (Full)", "Group LARS", "Group Garrote"],
    "MSE": [mse_ols, mse_lars, mse_garrote]
})

print("\nModel Comparison:")
print(results_df)



Model Comparison:
          Method       MSE
0     OLS (Full)  0.485308
1     Group LARS  2.301095
2  Group Garrote  1.548278


In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

# 1. DATA GENERATION — MODEL I
np.random.seed(0)
n_samples = 50
n_factors = 15
rho = 0.5

cov_matrix = rho ** np.abs(np.subtract.outer(np.arange(n_factors), np.arange(n_factors)))
Z = np.random.multivariate_normal(mean=np.zeros(n_factors), cov=cov_matrix, size=n_samples)

Z_discrete = np.zeros_like(Z, dtype=int)
for i in range(n_factors):
    q1, q2 = np.percentile(Z[:, i], [33.33, 66.67])
    Z_discrete[:, i] = np.digitize(Z[:, i], bins=[q1, q2])

def simulate_response(Zd):
    y = (
        1.8 * (Zd[:, 0] == 1) - 1.2 * (Zd[:, 0] == 0) +
        1.0 * (Zd[:, 2] == 1) + 0.5 * (Zd[:, 2] == 0) +
        1.0 * (Zd[:, 4] == 1) + 1.0 * (Zd[:, 4] == 0)
    )
    noise = np.random.normal(0, np.std(y), size=y.shape)
    return y + noise

Y = simulate_response(Z_discrete)

encoder = OneHotEncoder(drop='first', sparse_output=False)
X = encoder.fit_transform(Z_discrete)
X -= X.mean(axis=0)
Y -= Y.mean()

group_sizes = [2] * n_factors
group_indices = []
start = 0
for size in group_sizes:
    group_indices.append(np.arange(start, start + size))
    start += size

# 2. METHODS

def group_lasso(X, y, groups, lambda_=0.5, max_iter=100, tol=1e-6):
    beta = np.zeros(X.shape[1])
    for iteration in range(max_iter):
        beta_old = beta.copy()
        for j, g in enumerate(groups):
            # Oblicz resztę z pominięciem bieżącej grupy
            residual = y - X @ beta + X[:, g] @ beta[g]
            S_j = X[:, g].T @ residual
            norm_Sj = np.linalg.norm(S_j)
            threshold = lambda_ * np.sqrt(len(g))

            # Zgodnie z Yuan & Lin (2006) - eq. (2.4)
            if norm_Sj <= threshold or np.isnan(norm_Sj):
                beta[g] = 0
            else:
                shrinkage = 1 - threshold / norm_Sj
                beta[g] = shrinkage * S_j
        if np.linalg.norm(beta - beta_old) < tol:
            break
    return beta, X @ beta




def group_lars(X, y, groups, max_groups=None):
    residual = y.copy()
    selected_groups = []
    beta = np.zeros(X.shape[1])
    for _ in range(max_groups or len(groups)):
        correlations = [
            np.linalg.norm(X[:, g].T @ residual) / len(g) if j not in selected_groups else -np.inf
            for j, g in enumerate(groups)
        ]
        best_group = np.argmax(correlations)
        if correlations[best_group] == -np.inf:
            break
        selected_groups.append(best_group)
        X_sub = np.hstack([X[:, g] for j, g in enumerate(groups) if j in selected_groups])
        model = LinearRegression(fit_intercept=False).fit(X_sub, y)
        coef = model.coef_
        index = 0
        for j in selected_groups:
            g = groups[j]
            beta[g] = coef[index:index + len(g)]
            index += len(g)
        residual = y - X @ beta
    return beta, X @ beta

def group_garrote(X, y, groups, lam=0.5):
    ols = LinearRegression(fit_intercept=False).fit(X, y)
    beta_ols = ols.coef_
    d = np.ones(len(groups))
    for j, g in enumerate(groups):
        group_norm = np.linalg.norm(beta_ols[g])
        if group_norm > 0:
            shrink = max(0, 1 - lam * len(g) / group_norm)
            d[j] = shrink
        else:
            d[j] = 0
    beta_final = np.zeros_like(beta_ols)
    for j, g in enumerate(groups):
        beta_final[g] = beta_ols[g] * d[j]
    return beta_final, X @ beta_final

# 3. APPLY METHODS
beta_lasso, pred_lasso = group_lasso(X, Y, group_indices, lambda_=0.5)
print("NaNs in beta_lasso after fix:", np.isnan(beta_lasso).sum())


beta_lars, pred_lars = group_lars(X, Y, group_indices, max_groups=5)
beta_garrote, pred_garrote = group_garrote(X, Y, group_indices, lam=0.5)

print("NaNs in X:", np.isnan(X).sum())
print("NaNs in Y:", np.isnan(Y).sum())
print("NaNs in beta_lasso:", np.isnan(beta_lasso).sum())
print("NaNs in pred_lasso:", np.isnan(pred_lasso).sum())

if np.isnan(pred_lasso).any():
    print("Indexes with NaNs in pred_lasso:", np.where(np.isnan(pred_lasso))[0])


# 4. COMPARE MODELS
mse_lasso = mean_squared_error(Y, pred_lasso)
mse_lars = mean_squared_error(Y, pred_lars)
mse_garrote = mean_squared_error(Y, pred_garrote)

results_df = pd.DataFrame({
    "Method": ["Group Lasso", "Group LARS", "Group Garrote"],
    "MSE": [mse_lasso, mse_lars, mse_garrote]
})

print("\nModel Comparison:")
print(results_df)


NaNs in beta_lasso after fix: 0
NaNs in X: 0
NaNs in Y: 0
NaNs in beta_lasso: 0
NaNs in pred_lasso: 0

Model Comparison:
          Method            MSE
0    Group Lasso  2.696322e+102
1     Group LARS   2.301095e+00
2  Group Garrote   1.548278e+00


C:\Users\User\AppData\Local\Temp\ipykernel_19316\380460533.py:52: RuntimeWarning: overflow encountered in matmul
  residual = y - X @ beta + X[:, g] @ beta[g]
C:\Users\User\AppData\Local\Temp\ipykernel_19316\380460533.py:53: RuntimeWarning: overflow encountered in matmul
  S_j = X[:, g].T @ residual
C:\Users\User\AppData\Local\Temp\ipykernel_19316\380460533.py:53: RuntimeWarning: invalid value encountered in matmul
  S_j = X[:, g].T @ residual
C:\Users\User\AppData\Local\Temp\ipykernel_19316\380460533.py:52: RuntimeWarning: invalid value encountered in matmul
  residual = y - X @ beta + X[:, g] @ beta[g]
C:\Users\User\AppData\Local\Temp\ipykernel_19316\380460533.py:52: RuntimeWarning: invalid value encountered in add
  residual = y - X @ beta + X[:, g] @ beta[g]


In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize

# 1. DATA GENERATION — MODEL I
np.random.seed(0)
n_samples = 50
n_factors = 15
rho = 0.5

cov_matrix = rho ** np.abs(np.subtract.outer(np.arange(n_factors), np.arange(n_factors)))
Z = np.random.multivariate_normal(mean=np.zeros(n_factors), cov=cov_matrix, size=n_samples)

Z_discrete = np.zeros_like(Z, dtype=int)
for i in range(n_factors):
    q1, q2 = np.percentile(Z[:, i], [33.33, 66.67])
    Z_discrete[:, i] = np.digitize(Z[:, i], bins=[q1, q2])

def simulate_response(Zd):
    y = (
        1.8 * (Zd[:, 0] == 1) - 1.2 * (Zd[:, 0] == 0) +
        1.0 * (Zd[:, 2] == 1) + 0.5 * (Zd[:, 2] == 0) +
        1.0 * (Zd[:, 4] == 1) + 1.0 * (Zd[:, 4] == 0)
    )
    noise = np.random.normal(0, np.std(y), size=y.shape)
    return y + noise

Y = simulate_response(Z_discrete)

# One-hot encoding + centrowanie + normalizacja kolumn
encoder = OneHotEncoder(drop='first', sparse_output=False)
X = encoder.fit_transform(Z_discrete)
X -= X.mean(axis=0)
X = normalize(X, axis=0)
Y -= Y.mean()

# Grupy
group_sizes = [2] * n_factors
group_indices = []
start = 0
for size in group_sizes:
    group_indices.append(np.arange(start, start + size))
    start += size

# 2. METHODS

def group_lasso(X, y, groups, lambda_=0.05, max_iter=100, tol=1e-6):
    beta = np.zeros(X.shape[1])
    for iteration in range(max_iter):
        beta_old = beta.copy()
        for j, g in enumerate(groups):
            residual = y - X @ beta + X[:, g] @ beta[g]
            S_j = X[:, g].T @ residual
            norm_Sj = np.linalg.norm(S_j)
            threshold = lambda_ * np.sqrt(len(g))
            if norm_Sj <= threshold or np.isnan(norm_Sj):
                beta[g] = 0
            else:
                shrinkage = 1 - threshold / norm_Sj
            beta_g_update = shrinkage * S_j
            if np.linalg.norm(beta_g_update) > 1e3:  # limit stabilizujący
                beta[g] = 0
            else:
                beta[g] = beta_g_update

        if np.linalg.norm(beta - beta_old) < tol:
            break
    return beta, X @ beta

def group_lars(X, y, groups, max_groups=None):
    residual = y.copy()
    selected_groups = []
    beta = np.zeros(X.shape[1])
    for _ in range(max_groups or len(groups)):
        correlations = [
            np.linalg.norm(X[:, g].T @ residual) / len(g) if j not in selected_groups else -np.inf
            for j, g in enumerate(groups)
        ]
        best_group = np.argmax(correlations)
        if correlations[best_group] == -np.inf:
            break
        selected_groups.append(best_group)
        X_sub = np.hstack([X[:, g] for j, g in enumerate(groups) if j in selected_groups])
        model = LinearRegression(fit_intercept=False).fit(X_sub, y)
        coef = model.coef_
        index = 0
        for j in selected_groups:
            g = groups[j]
            beta[g] = coef[index:index + len(g)]
            index += len(g)
        residual = y - X @ beta
    return beta, X @ beta

def group_garrote(X, y, groups, lam=0.5):
    ols = LinearRegression(fit_intercept=False).fit(X, y)
    beta_ols = ols.coef_
    d = np.ones(len(groups))
    for j, g in enumerate(groups):
        group_norm = np.linalg.norm(beta_ols[g])
        if group_norm > 0:
            shrink = max(0, 1 - lam * len(g) / group_norm)
            d[j] = shrink
        else:
            d[j] = 0
    beta_final = np.zeros_like(beta_ols)
    for j, g in enumerate(groups):
        beta_final[g] = beta_ols[g] * d[j]
    return beta_final, X @ beta_final

# 3. APPLY METHODS
beta_lasso, pred_lasso = group_lasso(X, Y, group_indices, lambda_=0.05)
beta_lars, pred_lars = group_lars(X, Y, group_indices, max_groups=5)
beta_garrote, pred_garrote = group_garrote(X, Y, group_indices, lam=0.5)

# 4. CHECK NaNs
print("NaNs in X:", np.isnan(X).sum())
print("NaNs in Y:", np.isnan(Y).sum())
print("NaNs in beta_lasso:", np.isnan(beta_lasso).sum())
print("NaNs in pred_lasso:", np.isnan(pred_lasso).sum())

# 5. COMPARE MODELS
mse_lasso = mean_squared_error(Y, pred_lasso)
mse_lars = mean_squared_error(Y, pred_lars)
mse_garrote = mean_squared_error(Y, pred_garrote)

results_df = pd.DataFrame({
    "Method": ["Group Lasso", "Group LARS", "Group Garrote"],
    "MSE": [mse_lasso, mse_lars, mse_garrote]
})

print("\nModel Comparison:")
print(results_df)


NaNs in X: 0
NaNs in Y: 0
NaNs in beta_lasso: 0
NaNs in pred_lasso: 0

Model Comparison:
          Method           MSE
0    Group Lasso  24789.511559
1     Group LARS      2.301095
2  Group Garrote      0.683081


In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# 1. DATA GENERATION — MODEL I
np.random.seed(0)
n_samples = 50
n_factors = 15
rho = 0.5

cov_matrix = rho ** np.abs(np.subtract.outer(np.arange(n_factors), np.arange(n_factors)))
Z = np.random.multivariate_normal(mean=np.zeros(n_factors), cov=cov_matrix, size=n_samples)

Z_discrete = np.zeros_like(Z, dtype=int)
for i in range(n_factors):
    q1, q2 = np.percentile(Z[:, i], [33.33, 66.67])
    Z_discrete[:, i] = np.digitize(Z[:, i], bins=[q1, q2])

def simulate_response(Zd):
    y = (
        1.8 * (Zd[:, 0] == 1) - 1.2 * (Zd[:, 0] == 0) +
        1.0 * (Zd[:, 2] == 1) + 0.5 * (Zd[:, 2] == 0) +
        1.0 * (Zd[:, 4] == 1) + 1.0 * (Zd[:, 4] == 0)
    )
    noise = np.random.normal(0, np.std(y), size=y.shape)
    return y + noise

Y = simulate_response(Z_discrete)
Y -= Y.mean()

encoder = OneHotEncoder(drop='first', sparse_output=False)
X = encoder.fit_transform(Z_discrete)
X -= X.mean(axis=0)
X /= np.linalg.norm(X, axis=0)  # normalizacja kolumn

group_sizes = [2] * n_factors
group_indices = []
start = 0
for size in group_sizes:
    group_indices.append(np.arange(start, start + size))
    start += size

# 2. YOUR ORIGINAL GROUP LASSO
def group_lasso(X, y, groups, lambda_, max_iter=100, tol=1e-6):
    beta = np.zeros(X.shape[1])
    for _ in range(max_iter):
        beta_old = beta.copy()
        for j, g in enumerate(groups):
            residual = y - X @ beta + X[:, g] @ beta[g]
            S_j = X[:, g].T @ residual
            norm_Sj = np.linalg.norm(S_j)
            threshold = lambda_ * np.sqrt(len(g))
            if norm_Sj <= threshold or np.isnan(norm_Sj):
                beta[g] = 0
            else:
                shrinkage = 1 - threshold / norm_Sj
                beta_g_update = shrinkage * S_j
                if np.linalg.norm(beta_g_update) > 1e3:
                    beta[g] = 0
                else:
                    beta[g] = beta_g_update
        if np.linalg.norm(beta - beta_old) < tol:
            break
    return beta, X @ beta

# 3. CROSS-VALIDATION FOR LAMBDA SELECTION
kf = KFold(n_splits=5, shuffle=True, random_state=42)
lambdas = np.logspace(-2, 0.5, 20)  # od 0.01 do ~3.16

best_lambda = None
best_mse = np.inf

print("Performing cross-validation for group_lasso...")

for lam in lambdas:
    mses = []
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]

        beta, _ = group_lasso(X_train, y_train, group_indices, lambda_=lam)
        y_pred = X_test @ beta
        if np.any(np.isnan(y_pred)):
            mse = np.inf
        else:
            mse = mean_squared_error(y_test, y_pred)
        mses.append(mse)

    mean_mse = np.mean(mses)
    print(f"λ = {lam:.4f} | CV MSE = {mean_mse:.4f}")
    if mean_mse < best_mse:
        best_mse = mean_mse
        best_lambda = lam

print(f"\n✅ Best lambda: {best_lambda:.4f} | CV MSE: {best_mse:.4f}")

# 4. FIT FINAL MODEL
beta_final, pred_final = group_lasso(X, Y, group_indices, lambda_=best_lambda)
mse_final = mean_squared_error(Y, pred_final)

print(f"\n📊 Final Group Lasso MSE on full data: {mse_final:.4f}")


Performing cross-validation for group_lasso...
λ = 0.0100 | CV MSE = 124.8265
λ = 0.0135 | CV MSE = 119.7698
λ = 0.0183 | CV MSE = 113.1176
λ = 0.0248 | CV MSE = 104.4650
λ = 0.0336 | CV MSE = 93.3944
λ = 0.0455 | CV MSE = 79.5809
λ = 0.0616 | CV MSE = 63.0123
λ = 0.0834 | CV MSE = 44.3725
λ = 0.1129 | CV MSE = 25.5574
λ = 0.1528 | CV MSE = 11.1446
λ = 0.2069 | CV MSE = 4.5060
λ = 0.2801 | CV MSE = 3.2148
λ = 0.3793 | CV MSE = 2.9620
λ = 0.5135 | CV MSE = 2.7088
λ = 0.6952 | CV MSE = 2.5939
λ = 0.9412 | CV MSE = 2.5447
λ = 1.2743 | CV MSE = 2.5458
λ = 1.7252 | CV MSE = 2.5960
λ = 2.3357 | CV MSE = 2.7034
λ = 3.1623 | CV MSE = 2.7813

✅ Best lambda: 0.9412 | CV MSE: 2.5447

📊 Final Group Lasso MSE on full data: 1.1549
